<a id='top'></a>
# A Quantum Approach for Portfolio Optimization #

## Table of Contents
1. [Introduction](#introduction)
2. [Portfolio Optimization Problem](#prob_description)
    <br>2.a. [Classical Solution Using OR-Tools (TBD!!!)](#classical_solution_using_OR)
    <br>2.b. [Creating the QUBO representation](#qubo)
3. [Quantum computing solution using the QC Ware API](#quantum_solution)
    <br>3.a. [Execution via IBM's Qiskit software simulator using the QC Ware API](#qiskit_software_simulator)
    <br>3.b. [Execution via Google's software simulator using the QC Ware API](#google_software_simulator)
    <br>3.c. [Execution via D-Wave Systems' Quantum Annealer using the QC Ware API](#dwave_quantum_annealer)
    <br>3.d. [Execution via a classical brute force solver](#classical_brute_solver)
  

# 1. Introduction <a id="introduction"></a>

Portfolio optimization is the problem of optimally allocating a budget to a collection of assets. Real world examples can be found throughout the stock market world: groups and individuals want to invest their money in order to maximize profit and minimize risk.  Here, "optimal" can mean a number of different things.  For example, one could define "optimal" to mean that the expected value of the return is maximized, though in practice this naive approach is typically not very useful. Ultimately, "optimality" is precisly definied via an objective function, which we cover in more detail in the next section.

One widely used definition of optimality comes from 
<a href="https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf" style="text-decoration: none;">Markowitz</a>.  Informally speaking, Markowitz proposed that an allocation is optimal whenever the corresponding expected return is maximized <em>and</em> variance is minimized.  In other words, one seeks to maximize gains, while at the same time minimize risk. The following diagram will give you a sense of how we'll be approaching this problem.

![](https://files.slack.com/files-pri/T24940PQV-FHDQYHM6U/problem_setup.png?pub_secret=2c46907453)

# 2. Portfolio Optimization Problem <a id="prob_description"></a>

While there are many different ways of formally modelling the problem, here we focus on a quadratic unconstrained binary optimization (QUBO) model. This allows us to run the problem both on universal quantum computers as well as on special purpose quantum annealers. 

For Operations Research professionals wondering how to formulate their problems as a QUBO, the paper <a href="https://arxiv.org/pdf/1302.5843.pdf" style="text-decoration: none;">Ising formulations of many NP problems</a>, by Andrew Lucas, 2014, gives <em>Ising</em> formulations of many NP problems. Given that Ising and QUBO problems are equivalent under the appropriate transformation, one can in turn use these to obtain the QUBO forms.

Let's formally define the problem. Suppose the decision maker has a total budget $B$ she must allocate among $n$ assets. For each asset, $i$, we assume she has the option to allocate either some non-zero quantity $B_i>0$, or $0$. In this set-up, her choice is a binary one: either buy a $B_i$ of the asset or none at all. Because we can't invest more money than we have, we know that

$$
\begin{equation}
\sum_{i=1}^{n}x_{i}B_{i} \leq B
\end{equation}
$$ 

We can use $R_i$ to denote the return from asset $i$ if a budget of $B_i$ is allocated towards this asset.  So, $R$ represents the return from the entire portfolio.  In other words,

$$
\begin{equation}
R=\sum_{i=1}^{n}R_{i}
\end{equation}
$$

Therefore, the expected return from the portfolio is

$$
\begin{equation}\label{eq:expected_return}\tag{1}
\text{E}(R)=\sum_{i=1}^{n}\text{E}(R_{i})
\end{equation}
$$

Next, consider the variance of $R$, which is given by

$$
\begin{equation}\label{eq:variance}\tag{2}
\text{Var}(R)=\sum_{i,j=1}^{n}\text{Cov}(R_{i},R_{j})
\end{equation}
$$

where $\text{Cov}(R_{i},R_{j})$ is the co-variance of the random variables $R_{i}$ and $R_{j}$. In this case the covariance between assets can be thought of as a measure of the risk of the portfolio. 

In the Markowitz model, one seeks to maximize equation $\eqref{eq:expected_return}$ while minimizing $\eqref{eq:variance}$.

Typically, this problem is expressed with the following formulation:

$$
\begin{equation}
\begin{split}
\min_{x_{i}\in{\{0,1\}}} \;\; & \sum_{i=1}^{n}-x_{i}\text{E}(R_{i}) 
\;\; +\;&\theta\sum_{i,j=1}^{n}x_{i}x_{j}\text{Cov}(R_{i},R_{j})
\end{split}
\end{equation}
$$

subject to

$$
\begin{equation}
\sum_{i=1}^{n}x_{i}B_{i} \leq B
\end{equation}
$$

$$
\begin{equation}
x_{i}\in\{0,1\}
\end{equation}
$$

## 2.a Classical Solution Using OR-Tools (TBD!!!) <a id="classical_solution_using_OR"></a>

<div id="qubo" />

## 2.b Creating the QUBO representation

We're now ready to convert this into a QUBO problem. Our QUBO problem will look remarkably similar but will include the addition of another term not typically seen in the classical formulation:

$$
\begin{equation}
\begin{split}
\min_{x_{i}\in{\{0,1\}}} \;\; & \theta_{1}\sum_{i=1}^{n}-x_{i}\text{E}(R_{i}) \\
\;\;  +\;&\theta_{2}\sum_{i,j=1}^{n}x_{i}x_{j}\text{Cov}(R_{i},R_{j})  \\
 +\;&\theta_{3}\left(\sum_{i=1}^{n}x_{i}B_{i}-B\right)^{2}
\end{split}
\end{equation}
$$

The parameters $0\leq\theta_{1},\theta_{2},\theta_{3}<\infty$ represent the relative importance of each term to the decision maker, and she is free to change these parameters to best reflect that (we'll give some examples of this in the next paragraph).  The first term in the objective function represents the expected return, i.e. the gain.  The second term represents the variance in the return, i.e. the risk.  Finally, the last term penalizes our decision maker when the sum of all $B_i$ is lower than the total available budget $B$.

The parameters $\theta_{1},\theta_{2}$, and $\theta_{3}$ represent the relative importance to the decision maker of each term, and she is free to change these parameters to best reflect that.  For instance, in the extreme case that the decision maker does not care about risk, but only about the possible gains, then she may set $\theta_{1}\gg\theta_{2}$.  On the other hand, if she is very risk averse she could set $\theta_{2}\gg\theta_{1}$. Finally,  $\theta_{3}$ can be adjusted to modulate the penalty of allocations which involve investing a total amount that is less than $B$.

We are now ready to start writing the code we'll use to approach this problem.  We start by importing the native python module `random`, which provides the functionality to generate pseudo-random numbers. This allows us to generate a random dataset that we will use to generate results.  We intentionally seed the random number generator so that the results are the same between runs (you can change the seed to try new values!)

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('portfolio_optimization.ipynb')

import random
random.seed(42)

Next, we set the values of the free parameters $\theta_{1},\theta_{2},\theta_{3}$; recall that these parameters were introduced in the section immediately following the QUBO problem statement.

In [2]:
theta1=1
theta2=1
theta3=1
print("theta1={0}".format(theta1))
print("theta2={0}".format(theta2))
print("theta3={0}".format(theta3))

theta1=1
theta2=1
theta3=1


Next, we choose the number of assets. For this very simple example we'll set the assets to 3, but feel free to generate a different number.

In [3]:
num_assets=3

For each of the assets, $i$, we will randomly generate the associated price, $B_i$. For simplicity, we define two positive integers `min_cost` and `max_cost`.  The prices will be chosen at random from the set of integers in the interval [`min_cost`,`max_cost`].

In [4]:
min_cost=1
max_cost=10
possible_costs=list(range(min_cost,max_cost+1))
costs={i:random.choice(possible_costs) for i in range(num_assets)}
print("Allowable asset allocation quantities: {0}".format(costs))

Allowable asset allocation quantities: {0: 2, 1: 1, 2: 5}


Next, we choose the total budget, $B$.

In [5]:
B=sum([costs[key] for key in costs])/2.
print("Total budget: {0}".format(B))

Total budget: 4.0


Next, we generate the expected returns, $\text{E}(R_i)$, associated to each asset.  Again, for simplicity, we define two positive integers `min_return` and `max_return`.  The expected returns will be chosen at random from the set of integers in the interval [`min_return`,`max_return`]. 

In [6]:
min_return=1
max_return=10
possible_returns=list(range(min_return,max_return+1))
E={(i,i):random.choice(possible_returns) for i in range(num_assets)}
print("Expected returns: {0}".format( {key[0]:E[key] for key in E}))

Expected returns: {0: 4, 1: 4, 2: 3}


Next, we generate the expected covariances $\text{Cov}(R_i,R_j)$ associated to each pair of assets.  We define two integers `min_covariance` and `max_covariance`, and set the prices at random from the set of integers in the interval [`min_covariance`,`max_covariance`]. 

In [7]:
Cov={}
min_covariance=-10
max_covariance=10
possible_covariances=list(range(min_covariance,max_covariance+1))
for i in range(num_assets):
    for j in range(i+1,num_assets):
        Cov[(i,j)]=random.choice(possible_covariances)
print("Covariances: {0}".format(Cov))

Covariances: {(0, 1): -7, (0, 2): 7, (1, 2): -8}


In this section we take care of the third term in the objective function.

In [8]:
penalty={}
for i in range(num_assets):
    penalty[(i,i)]=costs[i]*costs[i]-2*B*costs[i]
    for j in range(i+1,num_assets):
        penalty[(i,j)]=2*costs[i]*costs[j]
offset=theta3*B*B

Finally, we merge all the dictionaries we have created and form our final QUBO.

In [9]:
Q={}
for key in penalty:
    Q[key]=theta3*penalty[key]
for key in Cov:
    Q[key]+=theta2*Cov[key]
for key in E:
    Q[key]+=-theta1*E[key]

print("Final QUBO: {0}".format(Q))

Final QUBO: {(0, 0): -16.0, (0, 1): -3, (0, 2): 27, (1, 1): -11.0, (1, 2): 2, (2, 2): -18.0}


We can visualize the QUBO matrix that we've just made as the following:

$$
Q =
\begin{pmatrix}
E(R_{1}) & Cov(R_{1},R_{2}) & Cov(R_{1},R_{3})\\
0 & E(R_{2}) & Cov(R_{2},R_{3}) \\
0 & 0 & E(R_{3})
\end{pmatrix}
\;\;
$$

***
## 3. Quantum Computing Solution using the QC Ware API<a id="quantum_solution"></a>

Using QC Ware's API, we can solve this problem with the `optimize_binary()` function.  With the `optimize_binary()` function, the user has the option to choose from a number of different backend solvers.  Without having to change the way the QUBO is input, QC Ware's tools then automatically formulate the problem in a way that is suitable for the selected solver's corresponding software and hardware environment.  This allows the user to explore with minimal hassle the backend that might be most well-suited for her application.

Let us take a look at a few different solvers.  We start by importing the `qcware` module and entering your API key if needed (not necessary on Forge-hosted notebooks)

In [10]:
import qcware.types
import qubovert as qv

### 3.a Execution via a classical QAOA simulator using the QC Ware API <a id="qaoa_software_simulator"></a>

The first solver we explore is `qcware/cpu_simulator`, which is an implementation of the quantum approximate optimization aglorithm (QAOA) [<sup>2</sup>](https://arxiv.org/abs/1411.4028) for (classical) energy minimization problems of the Ising model, developed using QCWare's Quasar simulator.  We briefly recall that any Ising solver can also be used for QUBOs due to the mathematical equivalence between QUBOs and Ising problems under the appropriate transformation.  This in turns allows us to solve our QUBO formulation of the portfolio optimization problem.

The following call takes two arguments:
1. the `BinaryProblem` class, which represents our QUBO, and
2. a string, `qcware/cpu_simulator`, which is the name of the desired backend solver.

In [11]:
qubo = qv.QUBO(Q)
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
problem = qcware.types.optimization.BinaryProblem(objective=poly)

solver1='qcware/cpu_simulator'
response = forge.optimization.optimize_binary(instance=problem, backend=solver1)
print(response)

Objective value: -30
Solution: (1, 1, 0) (and 0 other equally good solutions)


### 3.b Execution via D-Wave Systems' Quantum Annealer using the QC Ware API <a id="dwave_quantum_annealer"></a>

While the above two solvers target two distinct universal quantum computing software frameworks, we can also target a special purpose quantum annealing hardware framework: the D-Wave 2000Q [<sup>6</sup>](https://www.dwavesys.com).  As an example, let us use the solver `dwave_hardware`, which is an implementation of the simulated annealing algorithm [<sup>7</sup>](https://docs.dwavesys.com/docs/latest/c_solver_2.html) for (classical) energy minimization problems of the Ising model, developed using D-Wave's Ocean [<sup>8</sup>](https://docs.dwavesys.com/docs/latest/index.html) software framework.

Again, as we can see below, despite the notably different architectures of universal quantum computers and special purpose quantum annealers, switching to this different backend is done simply by changing the value of the string corresponding desired backend solver.

In [12]:
qubo = qv.QUBO(Q)
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
problem = qcware.types.optimization.BinaryProblem(objective=poly)

solver3='dwave/2000q'
response = forge.optimization.optimize_binary(instance=problem, backend=solver3)
print(response)

Objective value: -30
Solution: (1, 1, 0)


### 3.c Execution via a classical brute force solver <a id="classical_brute_solver"></a>

As a fourth and final example, we briefly demonstrate the use of the `brute_force` solver.  This is a purely classical solver, and it solves the problem via the most primitive algorithm.  As the name suggests, this solver simply loops over all possible solutions, and keeps track of the configuration with the lowest energy.  Due to its primitive nature, this algorithm is limited to rather small problem sizes in practice.  On the other hand, because this algorithm returns the global optima with probability 1, it can be used with smaller problems as a debugging tool.

In [13]:
qubo = qv.QUBO(Q)
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
problem = qcware.types.optimization.BinaryProblem(objective=poly)

solver4='qcware/cpu'
response = forge.optimization.optimize_binary(instance=problem, backend=solver4)
print(response)

Objective value: -30
Solution: (1, 1, 0)


Note: If you are viewing this on GitHub and want to access QC Ware's API to execute the notebook you will need an API key.
<br><a href="#top">Back to Table of Contents</a>